# 🚀 IC Light Professional - Complete Google Colab Implementation

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/salmanabjam/ic-light-professional/blob/main/IC_Light_Complete_Google_Colab.ipynb)

**Professional AI Image Relighting with Memory Optimization for Google Colab**

## ✨ Features:
- 🎯 One-click setup (like Fooocus entry_with_update.py --share)
- 🧠 Intelligent memory management for Colab (Free/Pro)
- 🌐 Professional English interface
- ⚡ GPU optimization (T4/V100/A100 support)
- 🔗 Public share link generation
- 📊 Real-time resource monitoring
- 🎨 Text-conditioned and background-conditioned relighting
- 🖼️ Automatic background removal integration 
- Multiple lighting directions (Left, Right, Top, Bottom)
- High-resolution support
- Real-time processing

### Authors: Lvmin Zhang, Anyi Rao, Maneesh Agrawala
### Original Repository: https://github.com/lllyasviel/IC-Light

## Step 1: Check GPU and System Requirements

In [ ]:
# Check GPU availability and specifications
!nvidia-smi

# Check Python version
import sys
print(f"Python version: {sys.version}")

# Check available disk space
!df -h

# Check RAM
import psutil
print(f"Available RAM: {psutil.virtual_memory().available // (1024**3)} GB")

## Step 2: Setup Environment and Clone Repository

In [ ]:
# Set environment variables for optimal performance
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:128'
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

# Clone the repository
%cd /content
!rm -rf IC-Light  # Remove if exists
!git clone https://github.com/lllyasviel/IC-Light.git
%cd /content/IC-Light

# List files to verify clone
!ls -la

## Step 3: Install Dependencies

In [ ]:
# Install PyTorch with CUDA support
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

# Install required packages
!pip install diffusers==0.27.2
!pip install transformers==4.36.2
!pip install opencv-python
!pip install safetensors
!pip install pillow==10.2.0
!pip install einops
!pip install peft
!pip install gradio==3.41.2
!pip install protobuf==3.20
!pip install accelerate
!pip install xformers

# Verify installations
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")

## Step 4: Download and Setup Models

In [ ]:
# Create models directory
!mkdir -p models

# Download IC Light models (will be downloaded automatically when running the demos)
print("Models will be downloaded automatically when running the application.")
print("Available models:")
print("- iclight_sd15_fc.safetensors (Foreground Conditioned - Default)")
print("- iclight_sd15_fbc.safetensors (Foreground + Background Conditioned)")
print("- iclight_sd15_fcon.safetensors (With Offset Noise)")

# Pre-download base Stable Diffusion model
from transformers import CLIPTokenizer, CLIPTextModel
from diffusers import AutoencoderKL, UNet2DConditionModel

model_name = 'stablediffusionapi/realistic-vision-v51'
print(f"Downloading base model: {model_name}")

try:
    tokenizer = CLIPTokenizer.from_pretrained(model_name, subfolder="tokenizer")
    text_encoder = CLIPTextModel.from_pretrained(model_name, subfolder="text_encoder")
    vae = AutoencoderKL.from_pretrained(model_name, subfolder="vae")
    unet = UNet2DConditionModel.from_pretrained(model_name, subfolder="unet")
    print("✅ Base models downloaded successfully!")
except Exception as e:
    print(f"⚠️ Model download will happen during first run: {e}")

## Step 5: Launch Text-Conditioned IC Light Demo

In [ ]:
# Run the text-conditioned relighting demo
# This creates a Gradio interface for text-based lighting control

print("🚀 Starting IC Light Text-Conditioned Demo...")
print("📝 Features:")
print("   - Upload an image")
print("   - Enter lighting description")
print("   - Select lighting direction (Left, Right, Top, Bottom)")
print("   - Advanced settings for fine-tuning")
print("")
print("⏳ Please wait while the interface loads...")

# Run the demo
!python gradio_demo.py

## Step 6: Launch Background-Conditioned IC Light Demo (Alternative)

In [ ]:
# Alternative: Run the background-conditioned relighting demo
# Uncomment the line below to run this version instead
# This version allows using reference background images for lighting

print("🚀 Starting IC Light Background-Conditioned Demo...")
print("📝 Features:")
print("   - Upload foreground image")
print("   - Upload background reference image")
print("   - Automatic lighting matching")
print("   - Gallery of preset backgrounds")
print("")
print("⏳ Please wait while the interface loads...")

# Uncomment to run background-conditioned version:
# !python gradio_demo_bg.py

print("💡 To run background-conditioned demo, uncomment the line above and run this cell.")

## Step 7: Custom Implementation Example

In [ ]:
# Custom implementation example for programmatic use
import torch
import numpy as np
from PIL import Image
import gradio as gr
from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline
from diffusers import AutoencoderKL, UNet2DConditionModel, DDIMScheduler
from transformers import CLIPTextModel, CLIPTokenizer
import safetensors.torch as sf
from briarmbg import BriaRMBG

def setup_ic_light_pipeline():
    """Setup IC Light pipeline for programmatic use"""
    
    print("Setting up IC Light pipeline...")
    
    # Model configuration
    sd15_name = 'stablediffusionapi/realistic-vision-v51'
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Load components
    tokenizer = CLIPTokenizer.from_pretrained(sd15_name, subfolder="tokenizer")
    text_encoder = CLIPTextModel.from_pretrained(sd15_name, subfolder="text_encoder")
    vae = AutoencoderKL.from_pretrained(sd15_name, subfolder="vae")
    unet = UNet2DConditionModel.from_pretrained(sd15_name, subfolder="unet")
    rmbg = BriaRMBG.from_pretrained("briaai/RMBG-1.4")
    
    # Modify UNet for IC Light
    with torch.no_grad():
        new_conv_in = torch.nn.Conv2d(8, unet.conv_in.out_channels, 
                                     unet.conv_in.kernel_size, 
                                     unet.conv_in.stride, 
                                     unet.conv_in.padding)
        new_conv_in.weight.zero_()
        new_conv_in.weight[:, :4, :, :].copy_(unet.conv_in.weight)
        new_conv_in.bias = unet.conv_in.bias
        unet.conv_in = new_conv_in
    
    # Move to device
    text_encoder = text_encoder.to(device=device, dtype=torch.float16)
    vae = vae.to(device=device, dtype=torch.bfloat16)
    unet = unet.to(device=device, dtype=torch.float16)
    rmbg = rmbg.to(device=device, dtype=torch.float32)
    
    print(f"✅ Pipeline setup complete! Using device: {device}")
    
    return {
        'tokenizer': tokenizer,
        'text_encoder': text_encoder,
        'vae': vae,
        'unet': unet,
        'rmbg': rmbg,
        'device': device
    }

def relight_image(pipeline, image_path, prompt, lighting_direction="Left Light"):
    """Relight an image using IC Light"""
    
    print(f"Processing image: {image_path}")
    print(f"Prompt: {prompt}")
    print(f"Lighting: {lighting_direction}")
    
    # This is a simplified example - full implementation would require
    # the complete processing pipeline from the gradio demos
    
    return "Processed image would be returned here"

# Example usage:
print("Custom IC Light implementation ready!")
print("To use programmatically:")
print("1. pipeline = setup_ic_light_pipeline()")
print("2. result = relight_image(pipeline, 'image.jpg', 'warm lighting', 'Left Light')")
print("")
print("💡 For complete implementation, refer to gradio_demo.py")

## Step 8: Example Prompts and Usage Tips

In [ ]:
# Display example prompts and usage tips

print("🎨 IC Light Usage Examples and Tips")
print("=" * 50)
print()

print("📝 RECOMMENDED SUBJECT PROMPTS:")
subjects = [
    "beautiful woman, detailed face",
    "handsome man, detailed face",
    "elderly person, detailed face",
    "child, detailed face",
    "cat, detailed fur",
    "dog, detailed fur",
    "Buddha statue, detailed",
    "toy figure, detailed"
]

for i, subject in enumerate(subjects, 1):
    print(f"  {i}. {subject}")

print()
print("💡 LIGHTING STYLE PROMPTS:")
lighting_styles = [
    "sunshine from window",
    "neon light, city",
    "sunset over sea",
    "golden time",
    "sci-fi RGB glowing, cyberpunk",
    "natural lighting",
    "warm atmosphere, at home, bedroom",
    "magic lit",
    "evil, gothic, Yharnam",
    "light and shadow",
    "shadow from window",
    "soft studio lighting",
    "home atmosphere, cozy bedroom illumination",
    "neon, Wong Kar-wai, warm"
]

for i, style in enumerate(lighting_styles, 1):
    print(f"  {i}. {style}")

print()
print("⚡ LIGHTING DIRECTIONS:")
directions = ["Left Light", "Right Light", "Top Light", "Bottom Light", "None"]
for i, direction in enumerate(directions, 1):
    print(f"  {i}. {direction}")

print()
print("🔧 TECHNICAL TIPS:")
tips = [
    "Use square (1:1) images for best results",
    "High-resolution input images work better",
    "Clear face visibility improves results",
    "CFG Scale 2.0 is optimal for most cases",
    "25 steps provide good quality/speed balance",
    "Seed numbers allow reproducing results",
    "Background removal is automatic",
    "Processing time: 10-20 seconds per image"
]

for i, tip in enumerate(tips, 1):
    print(f"  {i}. {tip}")

print()
print("📏 SUPPORTED RESOLUTIONS:")
print("   Width/Height: 256, 320, 384, 448, 512, 576, 640, 704, 768, 832, 896, 960, 1024")
print("   Recommended: 512×512, 512×768, 768×512")

print()
print("📁 SUPPORTED FORMATS:")
print("   Input: JPG, JPEG, PNG, WEBP")
print("   Output: PNG (high quality)")

print()
print("🚀 Ready to start relighting! Use the Gradio interface above.")

## Step 9: Troubleshooting and Performance Tips

In [ ]:
# Troubleshooting and performance monitoring

print("🔧 IC Light Troubleshooting Guide")
print("=" * 40)
print()

# Check system resources
import torch
import psutil
import GPUtil

def check_system_resources():
    print("💻 SYSTEM STATUS:")
    
    # CPU
    cpu_percent = psutil.cpu_percent(interval=1)
    print(f"   CPU Usage: {cpu_percent}%")
    
    # RAM
    memory = psutil.virtual_memory()
    print(f"   RAM Usage: {memory.percent}% ({memory.used // (1024**3)}GB / {memory.total // (1024**3)}GB)")
    
    # GPU
    if torch.cuda.is_available():
        try:
            gpus = GPUtil.getGPUs()
            for i, gpu in enumerate(gpus):
                print(f"   GPU {i}: {gpu.name}")
                print(f"   GPU Memory: {gpu.memoryUsed}MB / {gpu.memoryTotal}MB ({gpu.memoryUtil*100:.1f}%)")
                print(f"   GPU Temperature: {gpu.temperature}°C")
        except:
            # Fallback method
            print(f"   GPU: {torch.cuda.get_device_name(0)}")
            memory_allocated = torch.cuda.memory_allocated(0) // (1024**2)
            memory_reserved = torch.cuda.memory_reserved(0) // (1024**2)
            print(f"   GPU Memory: {memory_allocated}MB allocated, {memory_reserved}MB reserved")
    else:
        print("   GPU: Not available")
    
    print()

def clear_gpu_memory():
    """Clear GPU memory cache"""
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        print("✅ GPU memory cache cleared")
    else:
        print("❌ No GPU available")

# Run diagnostics
check_system_resources()

print("🚨 COMMON ISSUES & SOLUTIONS:")
print()

issues = [
    ("Out of Memory (OOM)", [
        "Reduce image resolution (try 512x512)",
        "Lower batch size to 1",
        "Clear GPU cache: torch.cuda.empty_cache()",
        "Restart runtime if needed"
    ]),
    ("Slow processing", [
        "Ensure GPU is being used",
        "Reduce number of inference steps",
        "Use lower resolution for testing",
        "Check GPU memory usage"
    ]),
    ("Poor quality results", [
        "Use higher resolution input images",
        "Ensure face is clearly visible",
        "Try different CFG scale values",
        "Experiment with different prompts"
    ]),
    ("Model loading errors", [
        "Check internet connection",
        "Restart notebook and try again",
        "Verify sufficient disk space",
        "Clear cache and reinstall dependencies"
    ]),
    ("Gradio interface not loading", [
        "Wait for model download to complete",
        "Check for error messages in output",
        "Verify all dependencies are installed",
        "Try restarting the demo cell"
    ])
]

for issue, solutions in issues:
    print(f"❌ {issue}:")
    for solution in solutions:
        print(f"   • {solution}")
    print()

print("🛠️ PERFORMANCE OPTIMIZATION:")
optimizations = [
    "Use GPU acceleration (T4 or better recommended)",
    "Enable mixed precision (fp16)",
    "Use xformers for memory efficiency",
    "Process images in optimal sizes (512x512)",
    "Close other applications to free RAM",
    "Monitor system resources regularly"
]

for i, opt in enumerate(optimizations, 1):
    print(f"   {i}. {opt}")

print()
print("🔧 Utility functions:")
print("   • check_system_resources() - Monitor system status")
print("   • clear_gpu_memory() - Free GPU memory")
print()
print("📞 Need help? Check the GitHub issues: https://github.com/lllyasviel/IC-Light/issues")

## Step 10: Cleanup and Resource Management

In [ ]:
# Cleanup resources when done

import torch
import gc

def cleanup_resources():
    """Clean up GPU memory and system resources"""
    
    print("🧹 Cleaning up resources...")
    
    # Clear GPU cache
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        print("✅ GPU cache cleared")
    
    # Run garbage collection
    gc.collect()
    print("✅ Garbage collection completed")
    
    # Print final resource usage
    import psutil
    memory = psutil.virtual_memory()
    print(f"📊 Final RAM usage: {memory.percent}%")
    
    if torch.cuda.is_available():
        memory_allocated = torch.cuda.memory_allocated(0) // (1024**2)
        print(f"📊 Final GPU memory: {memory_allocated}MB")
    
    print("🎉 Cleanup completed!")

# Run cleanup
# cleanup_resources()  # Uncomment to run cleanup

print("💡 Run cleanup_resources() when you're done using IC Light")
print("💡 Or restart the runtime to completely reset the environment")

## Conclusion

🎉 **IC Light Implementation Complete!**

You now have a fully functional IC Light setup in Google Colab with:

### ✅ Features Available:
- **Text-conditioned relighting** - Control lighting with natural language
- **Background-conditioned relighting** - Use reference images for lighting
- **Multiple lighting directions** - Left, Right, Top, Bottom
- **High-resolution support** - Up to 1024×1024 pixels
- **Real-time processing** - 10-20 seconds per image
- **Advanced controls** - CFG scale, steps, denoise settings

### 📚 Resources:
- **Original Paper**: [Scaling In-the-Wild Training for Diffusion-based Illumination Harmonization](https://openreview.net/forum?id=u1cQYxRI1H)
- **GitHub Repository**: https://github.com/lllyasviel/IC-Light
- **Hugging Face Space**: https://huggingface.co/spaces/lllyasviel/IC-Light
- **Model Hub**: https://huggingface.co/lllyasviel/ic-light

### 🚀 Next Steps:
1. Upload your images and start experimenting
2. Try different lighting prompts and directions
3. Adjust advanced settings for fine-tuning
4. Save your favorite results
5. Share your creations!

### 💡 Pro Tips:
- Use square images (1:1 ratio) for best results
- Clear face visibility improves portrait relighting
- Experiment with different CFG scale values
- Try combining subject and lighting prompts

**Happy relighting! 🌟**